<a href="https://colab.research.google.com/github/rajibmondal/Projects.NLP/blob/master/Bangla_ULMFiT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai import *
from fastai.text import *

In [2]:
from google.colab import drive
drive.mount('/content/drive') #colab sourcerajueee012@gmail.com , data source and folder rajibmondal1218@gmail.com
root_dir = '/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/'
base_dir = root_dir + 'weights/VIET_NLP'

# base_dir = Path(base_dir)

# # check if the dirctory exists if not make it
# import os
# if not (os.path.exists(base_dir)):
#   base_dir.mkdir(parents=True, exist_ok=True)
# base_dir

# path = Path('/content/drive/My Drive/Project/Vietnamese_sentiment_analysis/')
# path.ls()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#from fastai import *
from fastai.text import *
folder = 'Bangla'

path = Path('/content/gdrive/My Drive/NLP/NLP_DATA/Bangla_dataset')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [4]:
path.ls()

[PosixPath('/content/gdrive/My Drive/NLP/NLP_DATA/Bangla_dataset/Bangla')]

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [0]:
bs = 128

In [0]:
data_path = Config.data_path()

In [0]:
#lang = 'vi'
#lang = 'zh'
lang = 'bn'

In [0]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [0]:
from fastai.basics import *
import re


def get_wiki(path,lang):
    name = f'{lang}wiki'
    if (path/name).exists():
        print(f"{path/name} already exists; not downloading")
        return

    xml_fn = f"{lang}wiki-latest-pages-articles.xml"
    zip_fn = f"{xml_fn}.bz2"

    if not (path/xml_fn).exists():
        print("downloading...")
        download_url(f'https://dumps.wikimedia.org/{name}/latest/{zip_fn}', path/zip_fn)
        print("unzipping...")
        bunzip(path/zip_fn)

    with working_directory(path):
        if not (path/'wikiextractor').exists(): os.system('git clone https://github.com/attardi/wikiextractor.git')
        print("extracting...")
        os.system("python wikiextractor/WikiExtractor.py --processes 4 --no_templates " +
            f"--min_text_length 4000 --filter_disambig_pages --log_file log -b 100G -q {xml_fn}")
    shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
    shutil.rmtree(path/'text')


def split_wiki(path,lang):
    dest = path/'docs'
    name = f'{lang}wiki'
    if dest.exists():
        print(f"{dest} already exists; not splitting")
        return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'<doc id="\d+" url="https://{lang}.wikipedia.org/wiki\?curid=\d+" title="([^"]+)">')
    lines = (path/name).open()
    f=None

    for i,l in enumerate(lines):
        if i%100000 == 0: print(i)
        if l.startswith('<doc id="'):
            title = title_re.findall(l)[0].replace('/','_')
            if len(title)>150: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w')
        else: f.write(l)
    f.close()
    return dest

In [11]:
get_wiki(path,lang)

downloading...


unzipping...
extracting...


In [12]:
path.ls()

[PosixPath('/root/.fastai/data/bnwiki/wikiextractor'),
 PosixPath('/root/.fastai/data/bnwiki/bnwiki'),
 PosixPath('/root/.fastai/data/bnwiki/log'),
 PosixPath('/root/.fastai/data/bnwiki/bnwiki-latest-pages-articles.xml'),
 PosixPath('/root/.fastai/data/bnwiki/bnwiki-latest-pages-articles.xml.bz2')]

In [13]:
!head -n4 {path}/{name}

<doc id="608" url="https://bn.wikipedia.org/wiki?curid=608" title="বাংলা ভাষা">
বাংলা ভাষা

বাংলা ভাষা (বাঙলা, বাঙ্গলা, তথা বাঙ্গালা নামগুলোতেও পরিচিত) একটি ইন্দো-আর্য ভাষা, যা দক্ষিণ এশিয়ার বাঙালি জাতির প্রধান কথ্য ও লেখ্য ভাষা। মাতৃভাষীর সংখ্যায় বাংলা ইন্দো-ইউরোপীয় ভাষা পরিবারের চতুর্থ ও বিশ্বের ষষ্ঠ বৃহত্তম ভাষা। মোট ব্যবহারকারীর সংখ্যা অনুসারে বাংলা বিশ্বের সপ্তম বৃহত্তম ভাষা। বাংলা সার্বভৌম ভাষাভিত্তিক জাতিরাষ্ট্র বাংলাদেশের একমাত্র রাষ্ট্রভাষা তথা সরকারি ভাষা এবং ভারতের পশ্চিমবঙ্গ, ত্রিপুরা, আসামের বরাক উপত্যকার সরকারি ভাষা। বঙ্গোপসাগরে অবস্থিত আন্দামান দ্বীপপুঞ্জের প্রধান কথ্য ভাষা বাংলা। এছাড়া ভারতের ঝাড়খণ্ড, বিহার, মেঘালয়, মিজোরাম, উড়িষ্যা রাজ্যগুলোতে উল্লেখযোগ্য পরিমাণে বাংলাভাষী জনগণ রয়েছে। ভারতে হিন্দির পরেই সর্বাধিক প্রচলিত ভাষা বাংলা। এছাড়াও মধ্য প্রাচ্য, আমেরিকা ও ইউরোপে উল্লেখযোগ্য পরিমাণে বাংলাভাষী অভিবাসী রয়েছে। সারা বিশ্বে সব মিলিয়ে ২৬ কোটির অধিক লোক দৈনন্দিন জীবনে বাংলা ব্যবহার করে। বাংলাদেশের জাতীয় সঙ্গীত এবং ভারতের জাতীয় সঙ্গীত ও স্তোত্র বাংলাতে রচিত।

In [14]:
dest = split_wiki(path,lang)

0
100000
200000
300000


In [15]:
dest.ls()[:5]

[PosixPath('/root/.fastai/data/bnwiki/docs/পাবলিক নলেজ প্রজেক্ট.txt'),
 PosixPath('/root/.fastai/data/bnwiki/docs/মিউনিখ বিমান দুর্ঘটনা.txt'),
 PosixPath('/root/.fastai/data/bnwiki/docs/পৌষ সংক্রান্তি.txt'),
 PosixPath('/root/.fastai/data/bnwiki/docs/ল্যেভ তল্\u200cস্তোয়.txt'),
 PosixPath('/root/.fastai/data/bnwiki/docs/অমল কুমার রায়চৌধুরী.txt')]

In [16]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

In [27]:
data.show_batch()

idx,text
0,"যমুনা নামে নামকরণ করা হয়েছে । \n xxunk ছাত্রদের জন্য চারটি লিফট , একটি সুসজ্জিত xxunk , একটি ডিপার্টমেন্টাল স্টোর , দুটি পাঠকক্ষ , একটি কম্পিউটার ল্যাব , একটি কাউন্সেলিং রুম , হল সংসদ কক্ষ ( স্থাপিত ২০১৯ ) , একটি সেলুন , একটি সুবিশাল হল অডিটোরিয়াম ও দূরদর্শন কক্ষ , xxunk হল শাখার রুম , একটি ইনডোর xxunk ( বর্তমানে ঢাকা বিশ্ববিদ্যালয়ের সবচেয়ে বড় xxunk নামে খ্যাত )"
1,"পাম্প চালানোর জন্য ব্যবহার করে আসছে । \n \n চার্লস ফ্রান্সিস ব্রুস ( xxunk ১৯২৯ ) ১৮৮৭ - ৮৮ সালে একটি xxunk ( বায়ু টারবাইন ) নির্মাণ করেন যা একটা ব্যাটারী মাধ্যমে বিদ্যুতের উৎপাদনের জন্য xxunk xxunk xxunk । ১৮৯১ সালে ডেনিশ পদার্থবিজ্ঞানী এবং উদ্ভাবক পল লা xxunk প্রথম "" বায়ু xxunk "" পরীক্ষা করেন । ১৯০০ সালের কাছাকাছি তিনি বায়ু - চালিত xxunk প্লান্ট বিকাশ শুরু করেন ।"
2,এবং সেখানে ইসলামী শিক্ষাব্যবস্থা প্রতিষ্ঠায় সাহায্য করেন । ১৮৮৪ সালে তিনি ফ্রান্সের প্যারিস যান এবং আফগানির xxmaj the xxmaj xxunk xxmaj bond ( আল xxunk আল xxunk ) নামক ইসলামী বিপ্লবী ধারার পত্রিকার প্রকাশনায় যুক্ত হন । এই পত্রিকাটি ব্রিটিশ বিরোধী মনোভাবকে প্রাধান্য দিত । তিনি ব্রিটেন যান এবং উচ্চপদস্থ কর্মকর্তাদের সাথে মিশর ও সুদান বিষয়ে আলোচনা করেন । ১৮৮৫ সালে তিনি বৈরুতে ফিরে আসেন বিভিন্ন ধর্মীয় মতাবলম্বী পন্ডিতদের দ্বারা
3,"সালে ইসরায়েলী কমিউনিস্ট পার্টির পত্রিকা "" জো xxunk এক সাক্ষাৎকারে দারবিশ বলেছেন : "" সেই একটি রাত সবাইকে শরনার্থী বানিয়ে দেয় , xxunk আমি শরনার্থী ছিলাম । xxunk আমি শরনার্থী হয়ে আছি । "" এটা কোন কাব্যিক xxunk ছিল না । ইসরায়েলী রাষ্ট্রের প্রথম আদমশুমারিতে যে সব ফিলিস্তিনী অন্তর্ভুক্ত হয়নি , নতুন ইসরায়েলী সরকার তাদেরকে পরিচয় পত্র দেয়নি । তাদের চিহ্নিত করে অবৈধ অনুপ্রবেশকারী হিসেবে । ফলে xxunk মত অজস্র"
4,"চিঠিতে xxunk সাইটটির প্রচার করতে বারণ করা হয়েছিল , কারণ তিনি আমেরিকান লাইব্রেরি অ্যাসোসিয়েশনের একটি অনুষ্ঠানে সাইটটি নিয়ে কথা বলেছিলেন । এর পরিপ্রেক্ষিতে যথাযথ গবেষণা চাপা দেওয়ার জন্য ওই প্রকাশক গোষ্ঠীর কড়া নিন্দা করা হয় , এবং এই চিঠিটি"
